<a href="https://colab.research.google.com/github/nezhalahnech/game-of-life/blob/master/roadmark_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()   # اختاري ملف .zip من جهازك


KeyboardInterrupt: 

In [ ]:
!unzip /content/ROADMARK.zip -d /content/ROADMARK

In [ ]:
import os

for root, dirs, files in os.walk("/content/ROADMARK"):
    level = root.replace("/content/ROADMARK", "").count(os.sep)
    indent = " " * (4 * level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * (4 * (level + 1))
    for f in files[:5]:  # نوري غير أول 5 ملفات باش مايطولش
        print(f"{subindent}{f}")


In [ ]:
# install ultralytics (YOLOv8)
!pip install ultralytics

In [ ]:
import os

def clean_labels(labels_path, remove_ids=["0","1"]):  # hna 7t IDs dyal خطوط الطريق
    for filename in os.listdir(labels_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_path, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()
            new_lines = [line for line in lines if line.split()[0] not in remove_ids]
            with open(filepath, "w") as f:
                f.writelines(new_lines)

# Apply 3la train/valid/test
clean_labels("/content/ROADMARK/train/labels")
clean_labels("/content/ROADMARK/valid/labels")
clean_labels("/content/ROADMARK/test/labels")


In [ ]:
from ultralytics import YOLO

# Load model (yolov8n = nano, tu peux changer)
model = YOLO("yolov8n.pt")

# Start training
model.train(
    data="/content/ROADMARK/data.yaml",  # path dyal dataset config
    epochs=50,
    imgsz=640,
    batch=16,
    name="roadmark_cleaned"
)


In [ ]:
from ultralytics import YOLO

# Load model (yolov8n = nano, tu peux changer)
model = YOLO("yolov8n.pt")

# Start training
model.train(
    data="/content/ROADMARK/data.yaml",  # path dyal dataset config
    epochs=50,
    imgsz=640,
    batch=16,
    name="roadmark_cleaned"
)


In [ ]:
from ultralytics import YOLO

# Load trained model
model = YOLO("/content/runs/detect/roadmark_cleaned3/weights/best.pt")

# Evaluate on test set
metrics = model.val()
print(metrics)

In [ ]:
import yaml

dataset_yaml = {
    "path": "/content/ROADMARK",
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": ["arrow", "crosswalk", "stop", "bus"]
}

with open("roadmark.yaml", "w") as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print(open("roadmark.yaml").read())

In [ ]:
from ultralytics import YOLO

# load pretrained YOLOv8 model (small)
model = YOLO("yolov8s.pt")

# train on the cleaned dataset
model.train(data="roadmark.yaml", epochs=50, imgsz=640, batch=16)

In [ ]:
# Evaluate on validation/test set
metrics = model.val()

# Print metrics
print(metrics)